<a href="https://colab.research.google.com/github/arjunkpreman/Transfer-Learning-Project/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58889256/58889256 [==============================] - 3s 0us/step


In [4]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False


In [5]:
# Add new classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)  # num_classes is the number of classes in your task

In [6]:

# Create the transfer learning model
model = Model(inputs=base_model.input, outputs=predictions)

In [7]:

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Set up data generators for training and validation
train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 3015 images belonging to 10 classes.


In [9]:
val_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)
val_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 921 images belonging to 10 classes.


In [10]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    validation_data=val_generator,
    validation_steps=val_generator.samples // 32,
    epochs=5
)

Epoch 1/5
94/94 [==============================] - 2373s 25s/step - loss: 1.0660 - accuracy: 0.6731 - val_loss: 0.7486 - val_accuracy: 0.7366
Epoch 2/5
94/94 [==============================] - 19s 201ms/step - loss: 0.3358 - accuracy: 0.8874 - val_loss: 0.4086 - val_accuracy: 0.8583
Epoch 3/5
94/94 [==============================] - 20s 215ms/step - loss: 0.1823 - accuracy: 0.9470 - val_loss: 0.4113 - val_accuracy: 0.8583
Epoch 4/5
94/94 [==============================] - 20s 212ms/step - loss: 0.1164 - accuracy: 0.9692 - val_loss: 0.4070 - val_accuracy: 0.8627
Epoch 5/5
94/94 [==============================] - 20s 209ms/step - loss: 0.0764 - accuracy: 0.9849 - val_loss: 0.3276 - val_accuracy: 0.8962


In [17]:
# Save the trained model
model.save('/content/drive/MyDrive/Dataset/VGG16.h5')

In [18]:
model.evaluate(val_generator)

29/29 [==============================] - 4s 148ms/step - loss: 0.3196 - accuracy: 0.8990


[0.3195905387401581, 0.8990228176116943]